In [17]:
!pip install transformers

In [18]:
from transformers import AutoTokenizer, AutoModel
import torch

In [19]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

In [20]:
text = "hello world what a time to be alive!"
tokens = tokenizer.encode_plus(text, max_length=512,
                               truncation=True, padding='max_length',
                               return_tensors='pt')

Input IDs for CLS token is 101.

Input IDs for SEP token is 102.

Input IDs PAD Token is 0

we have input ids and segment ids.
segment ids also known as attention mask.

This tensor indicates which tokens should be attended to (value of 1) and which should be ignored (value of 0) during processing.

[pad] tokens are set as

In [21]:
tokens

{'input_ids': tensor([[ 101, 7592, 2088, 2054, 1037, 2051, 2000, 2022, 4142,  999,  102,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,

In [ ]:
text = "hello world what a time to be alive. My name is sayanteka"
tokens_new = tokenizer.encode_plus(text, max_length=512,
                               truncation=True, padding='max_length',
                               return_tensors='pt')



In [ ]:
tokens_new

{'input_ids': tensor([[  101,  7592,  2088,  2054,  1037,  2051,  2000,  2022,  4142,  1012,
          2026,  2171,  2003,  2360, 12956,  2912,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [22]:
outputs = model(**tokens)
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [23]:
#The dense vector representations of our text contained within 'last_hidden_state' tensor
embeddings = outputs.last_hidden_state
embeddings

tensor([[[ 0.3068, -0.0788,  1.7431,  ..., -0.0253, -0.1108,  0.0483],
         [ 0.7130,  0.1044,  1.8346,  ...,  0.1134, -0.0756,  0.1267],
         [ 0.8172,  0.1132,  1.5408,  ..., -0.3807,  0.0875, -0.1902],
         ...,
         [ 0.5351, -0.2527,  0.9820,  ..., -0.0215, -0.3696,  0.2446],
         [ 0.4957, -0.3423,  0.9106,  ..., -0.0352, -0.3554,  0.1440],
         [ 0.4143, -0.0859,  1.1123,  ...,  0.0122, -0.1843, -0.1364]]],
       grad_fn=<NativeLayerNormBackward0>)

In [24]:
embeddings.shape

torch.Size([1, 512, 768])

Sentence Embedding=Single vector encoding i.e. 512*768 will be changed to 768 in case of sentence embedding.

This is done using mean pool operation.

To do this mean pooling operation we will need to multiply each value in our embeddings tensor by it's respective attention_mask value - so that we ignore non-real tokens.

Attention mask value is our segment ids


In [25]:
attention_mask = tokens['attention_mask']  #segment ids
attention_mask.shape

torch.Size([1, 512])

In [28]:
attention_mask.unsqueeze(-1).shape #at -1 position 1 is added. if (-2) then it would have been [1,1,512]

torch.Size([1, 512, 1])

In [29]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([1, 512, 768])

In [32]:
mask[0].shape

torch.Size([512, 768])

In [34]:
mask[0][0].shape

torch.Size([768])

In [36]:
mask

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [41]:
mask[0]

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [42]:
mask[0][0]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [44]:
mask[0][-1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [40]:
len(mask[0][0]), len(mask[0]), len(mask)
#mask=1 batch=1 sntence
#mask[0]=512 tokens in 1 sentence where real tokens are 1 non real are 0
#mask[0][0]= since it is expanded for pooling operation where it is 1, it has been repeated 1 1 1 till 768 times for 1st token, similarly for 2nd token and so on

(768, 512, 1)

In [45]:
masked_embeddings = embeddings * mask
masked_embeddings.shape

torch.Size([1, 512, 768])

In [47]:
masked_embeddings

#in embeddings for non real tokens was not zero but in masked embeddings it is zero

tensor([[[ 0.3068, -0.0788,  1.7431,  ..., -0.0253, -0.1108,  0.0483],
         [ 0.7130,  0.1044,  1.8346,  ...,  0.1134, -0.0756,  0.1267],
         [ 0.8172,  0.1132,  1.5408,  ..., -0.3807,  0.0875, -0.1902],
         ...,
         [ 0.0000, -0.0000,  0.0000,  ..., -0.0000, -0.0000,  0.0000],
         [ 0.0000, -0.0000,  0.0000,  ..., -0.0000, -0.0000,  0.0000],
         [ 0.0000, -0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000]]],
       grad_fn=<MulBackward0>)

In [48]:
summed = torch.sum(masked_embeddings, 1) #summation along axis 1 so 512 removed (horizantally from left to right)
summed.shape

torch.Size([1, 768])

In [49]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9) #In mask  while summing min has beeen added in order to replace 0 by 1e-9 so that division by zero doesn't occur
summed_mask.shape

#Then sum the number of values that must be given attention in each position of the tensor along axis=1 so 512 removed

torch.Size([1, 768])

In [52]:
mean_pooled = summed / summed_mask

In [53]:
mean_pooled.shape

torch.Size([1, 768])

In [54]:
mean_pooled

tensor([[ 6.4371e-01, -1.6712e-01,  1.6938e+00,  1.0773e-01,  3.9651e-01,
         -1.0034e-01, -4.7002e-02,  7.9962e-01,  9.9557e-02, -9.4254e-01,
         -2.3453e-01, -2.0168e-02,  1.9673e-01,  3.9952e-01,  1.1500e-01,
          3.4273e-01, -6.4042e-01, -5.6996e-01,  2.7601e-01, -6.6139e-01,
         -7.0416e-01, -2.9305e-01, -1.5104e-01, -2.2795e-01,  9.0203e-01,
         -5.3868e-01, -1.0533e-01, -6.9518e-01, -3.7914e-01,  2.6069e-01,
         -8.8313e-01, -1.1394e-01,  1.0662e+00, -5.9388e-01, -4.5752e-01,
          1.4677e+00, -6.4333e-01, -1.7422e-01,  1.2722e-01, -4.2277e-01,
          6.6559e-01, -1.9796e-01,  1.1205e+00,  2.3012e-01, -1.3350e+00,
          4.1537e-01, -5.1203e-01,  4.1024e-01,  2.7452e-01, -8.5214e-01,
          1.2710e-01, -8.0763e-01, -6.3740e-01, -3.1229e-01, -4.9854e-01,
          3.2761e-01,  3.9804e-01, -4.4230e-01, -8.0382e-02,  4.9029e-01,
          6.1305e-01, -5.9101e-01,  5.5611e-01,  1.7230e-01, -1.0206e+00,
          1.7572e-01,  1.0949e+00, -3.